# Importing required libraries

In [5]:
import cv2,os
import numpy as np
from PIL import Image
import pickle
import sqlite3
import pandas as pd

# Using LBPH Face Recognizer for Face Recognition

In [6]:
rec=cv2.face.LBPHFaceRecognizer_create()
rec.read("Recognizer/TrainingData.yml")
faceCascade=cv2.CascadeClassifier('haarcascade_frontalface_default.xml');
path = 'DataSet'
df=pd.read_csv('ReservationDetails.csv')

# To get the details of the person detected from database

In [7]:
def getResDet(Id):
    conn=sqlite3.connect("TicketFaceRecog.db")
    cmd="SELECT * FROM Reservation WHERE ID="+str(Id)
    cursor=conn.execute(cmd)
    Det=0
    for row in cursor:
        Det=row
    conn.close()
    return Det

# Display the passenger details if ticket exists else no ticket 

In [9]:
# We have used both database and the CSV file to retrieve the passenger details.

cam = cv2.VideoCapture(0)
font_for = cv2.FONT_HERSHEY_SIMPLEX
thickness = 2
check = "Checked"

while(True):
    ret,img=cam.read();
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces=faceCascade.detectMultiScale(gray,1.3,5);
    for(x,y,w,h) in faces:
        cv2.rectangle(img,(x,y), (x+w, y+h), (0,0,0),3)
        id,conf=rec.predict(gray[y:y+h,x:x+w])
        Det = getResDet(id)
        if(Det==0):
            cv2.putText(img, "Status: No Ticket Found", (x-50,y+h+30),font_for,1,(0, 0, 255),thickness);
        else:
            rowss=-1
            conn=sqlite3.connect("TicketFaceRecog.db")
            Id=Det[0]
            cmd="UPDATE Reservation SET TicketChecked=' "+check+" ' WHERE ID="+str(Id)
            conn.execute(cmd)
            conn.commit()
            conn.close()
            cv2.putText(img, "Status: Ticket Booked", (x-80,y+h+30),font_for,1,(0, 220, 0),thickness);
            cv2.putText(img, "Name: " + str(Det[1]), (x-80,y+h+60),font_for,1,(0, 220, 0),thickness);
            for idss in df['PASSENGER UNIQUE ID']:
                rowss=rowss+1
                if int(idss)==id:
                    row_line=df.iloc[rowss]
        
                    cv2.putText(img, "Age: " + str(row_line[6]), (x-80,y+h+90),font_for,1,(0, 220, 0),thickness);
                    cv2.putText(img, "Gender: " + str(row_line[8]), (x-80,y+h+120),font_for,1,(0, 220, 0),thickness);
                    cv2.putText(img, "Journey Date: " + str(row_line[10]), (x-80,y+h+150),font_for,1,(0, 220, 0),thickness);
    cv2.imshow("Ticket Verification",img);
    if(cv2.waitKey(1)==ord('q')):
        break;
cam.release()
cv2.destroyAllWindows()

ValueError: cannot convert float NaN to integer